In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
# import seaborn as sns
import plotly
import plotly_express as px
import os
import plotly.io as pio
from floweaver import *
from ipysankeywidget import SankeyWidget

In [2]:
# Setting working directory
os.chdir ('C:\\Users\\KarlaC\\MAT-DP\\')

# Make folders for figures
if not os.path.exists('figures'):
    os.makedirs('figures')
    
if not os.path.exists('figures\countries'):
    os.makedirs('figures\countries')
if not os.path.exists('outputs'):
    os.makedirs('outputs')

In [ ]:
# for ne in ['E', 'W', 'K', 'R']:
#     globals()['df{}_country'.format(ne)] = pd.read_csv(r'outputs/df{}_total_bycountry.csv'.format(ne))
#     globals()['dfcountry_tech_{}'.format(ne)]= pd.read_csv(r'outputs/df{}_tech_bycountry.csv'.format(ne))
#     globals()['e_df{0}_countries_sep'.format(ne)]= pd.read_csv(r'outputs/errors{}_bymat_bycountry.csv'.format(ne))
#     globals()['e_df{0}c_tot'.format(ne)]= pd.read_csv(r'outputs/errors{}_total_bycountry.csv'.format(ne))
#     globals()['df{}c_tech_sep'.format(ne)]= pd.read_csv(r'outputs/{}_matbytech_bycountry.csv'.format(ne))


In [3]:
dfEc_tech_sep = pd.read_csv(r'outputs/E_matbytech_bycountry.csv')

dfEc_tech_sep_m = pd.melt(dfEc_tech_sep.drop(columns = 'Unnamed: 0'), 
                     id_vars = ['tech', 'Scenario','Country','Year'], 
                     var_name = 'Material', value_name = 'Value')
dfEc_tech_sep_m['Value'] = [x/1e12 for x in dfEc_tech_sep_m['Value']]

# W /1000
# K R /1e6
df_E = dfEc_tech_sep_m.rename(columns = {'tech':'Source'})
df_E['Target'] = 'Embodied emissions'#df_E['Material']


In [4]:
mat_country = pd.read_csv(r'outputs/massmat_bytech_bycountry.csv')

mat_country_m = pd.melt(mat_country, 
                     id_vars = ['tech', 'Scenario','Country','Year'], 
                     var_name = 'Material', value_name = 'Value')

df_M = mat_country_m.rename(columns = {'Material':'Target','tech':'Source'})
df_M['Material'] = df_M['Target']

In [5]:
df = pd.DataFrame(columns = ['Material','Source','Target','Value','Year','Scenario','Country'])

df = df.append(df_E)
df = df.append(df_M)


C:\Users\KarlaC\Anaconda3\lib\site-packages\pandas\core\frame.py:7123: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.




In [7]:
df.to_csv(r'outputs/sankey_data.csv',index=False)


In [8]:
#### DEFINE ALL SLICES IN ORDERING AS VARIABLES

materials = ['Aluminium', 'Bentonite', 'Carbon Fiber',
            'Cast Iron', 'Cement','Ceramics', 'Concrete','Copper', 'Epoxy', 'EVA ', 'Fibre Glass',
            'Glass', 'Lubricant', 'Non-Ferrous Metal', 'Paint', 'Plastic', 'PVC','Resin', 'Sand',
            'Silicon', 'Steel', 'Stainless Steel']

technologies = ['Wind (Onshore)', 'Wind (Offshore)',
       'Solar CSP', 'Solar PV', 'Hydro', 'Oil','Gas', 'Gas CCS', 'Nuclear',
       'Geothermal', 'Coal', 'Coal CCS', 'Biomass', 'BECCS', 'Hydrogen']

emissions = ['Material embodied emissions']


# Technologies to materials

In [27]:
partition_type = Partition.Simple('Material',df['Material'].unique())
nodes = {'technologies': ProcessGroup(technologies, Partition.Simple('process', technologies), title='Technologies'),
         'materials': ProcessGroup(materials, Partition.Simple('process', materials), title='Materials')
         }                         

ordering = [
    ['technologies'],
     ['materials']
    ]

bundles = [
    Bundle('technologies','materials')
]

colors = ['rgb(0.961, 1.034, 1.084)',
#           'rgb(245, 237, 226)',
        'rgb(179, 154, 154)',
        'rgb(220, 142, 106)',
        'rgb(232, 218, 120)',
        'rgb(162, 77, 55)',
        'rgb(187, 208, 182)',
        'rgb(61, 50, 25)',
        'rgb(195, 127, 120)',
        'rgb(108, 139, 124)',
        'rgb(183, 116, 87)',
        'rgb(255, 253, 228)',
        'rgb(155, 172, 162)',
        'rgb(213, 219, 139)',
        'rgb(32, 64, 79)',
        'rgb(172, 162, 151)',
        'rgb(222, 215, 199)',
        'rgb(213, 237, 231)',
        'rgb(87, 207, 197)',
        'rgb(91, 110, 95)',
        'rgb(251, 205, 198)',
        'rgb(209, 211, 212)',
        'rgb(182, 198, 226)'
            ]

# colors =['rgb(0.961, 1.034, 1.084)',
#  'rbg(0.702, 0.604, 0.604)',
#  '(0.863, 0.557, 0.416)',
# '(0.91, 0.855, 0.471)',
#  '(0.635, 0.302, 0.216)',
# '(0.733, 0.816, 0.714)',
# '(0.239, 0.196, 0.098)',
# '(0.765, 0.498, 0.471)',
#  '(0.424, 0.545, 0.486)',
# '(0.718, 0.455, 0.341)',
#  '(1.0, 0.992, 0.894)',
# '(0.608, 0.675, 0.635)',
#  '(0.835, 0.859, 0.545)',
#  '(0.125, 0.251, 0.31)',
# '(0.675, 0.635, 0.592)',
#  '(0.871, 0.843, 0.78)',
#  '(0.835, 0.929, 0.906)',
#  '(0.341, 0.812, 0.773)',
#  '(0.357, 0.431, 0.373)',
#  '(0.984, 0.804, 0.776)',
#  '(0.82, 0.827, 0.831)',
#  '(0.714, 0.776, 0.886)']

palette = pd.DataFrame(list(zip(materials,colors))).set_index(0)[1].to_dict()

sdd = SankeyDefinition(nodes, bundles, ordering,flow_partition=partition_type)
size = dict(width=1000, height=700)

In [125]:
# Floweaver can't save in loop
# df = df.rename(columns = {'Source':'source','Target':'target'})
# for c in ['UK','Uganda']:
#     for sc in df[df['Country']==c]['Scenario'].unique():
#         for y in df[(df['Country']==c)&(df['Scenario']==sc)]['Year'].unique():
#             dfc = df[(df['Country']==c)&(df['Scenario']==sc)&(df['Year']==y)]
#             print(c,sc, y)
#             Sank = weave(sdd, dfc, measures='Value', 
#                   palette=palette).to_widget(**size,margins=dict(left=100, right=120))
         

UK Baseline 2020
UK Consumer Transformation 2050
UK System Transformation 2050
UK Leading the Way 2050
UK Steady Progression 2050
Uganda Baseline 2010
Uganda Scenario 1 2040


In [33]:
df = df.rename(columns = {'Source':'source','Target':'target'})
c = 'Uganda'
sc = 'Baseline'
y = 2010
dfc = df[(df['Country']==c)&(df['Scenario']==sc)&(df['Year']==y)]
Sank = weave(sdd, dfc, measures='Value', 
      palette=palette).to_widget(**size,margins=dict(left=160, right=160))

In [34]:
Sank.auto_save_png(r'figures/countries/'+c+'_'+sc[:4]+str(y)+'TMSankey.png')

SankeyWidget(groups=[{'id': 'technologies', 'type': 'process', 'title': 'Technologies', 'nodes': ['technologie…

# Technologies to emissions

In [38]:
partition_type = Partition.Simple('Material',df['Material'].unique())
nodes = {'technologies': ProcessGroup(technologies, Partition.Simple('process', technologies), title='Technologies'),
        'emissions': ProcessGroup(emissions, Partition.Simple('process', emissions), title='Embodied Emissions')
         }                         

ordering = [
    ['technologies'],
     ['emissions']
    ]

bundles = [
    Bundle('technologies', 'emissions')
]


colors = ['rgb(0.961, 1.034, 1.084)',
#           'rgb(245, 237, 226)',
        'rgb(179, 154, 154)',
        'rgb(220, 142, 106)',
        'rgb(232, 218, 120)',
        'rgb(162, 77, 55)',
        'rgb(187, 208, 182)',
        'rgb(61, 50, 25)',
        'rgb(195, 127, 120)',
        'rgb(108, 139, 124)',
        'rgb(183, 116, 87)',
        'rgb(255, 253, 228)',
        'rgb(155, 172, 162)',
        'rgb(213, 219, 139)',
        'rgb(32, 64, 79)',
        'rgb(172, 162, 151)',
        'rgb(222, 215, 199)',
        'rgb(213, 237, 231)',
        'rgb(87, 207, 197)',
        'rgb(91, 110, 95)',
        'rgb(251, 205, 198)',
        'rgb(209, 211, 212)',
        'rgb(182, 198, 226)'
            ]

palette = pd.DataFrame(list(zip(materials,colors))).set_index(0)[1].to_dict()

sdd = SankeyDefinition(nodes, bundles, ordering,flow_partition=partition_type)
size = dict(width=1000, height=700)

In [21]:
for c in ['UK','Uganda']:
    for sc in df[df['Country']==c]['Scenario'].unique():
        for y in df[(df['Country']==c)&(df['Scenario']==sc)]['Year'].unique():
            dfc = df[(df['Country']==c)&(df['Scenario']==sc)&(df['Year']==y)]
            print(c,sc, y)

UK Baseline 2020
UK Consumer Transformation 2050


KeyboardInterrupt: 

In [39]:
df = df.rename(columns = {'Source':'source','Target':'target'})
c = 'Uganda'
sc = 'Scenario 1'
y = 2040
dfc = df[(df['Country']==c)&(df['Scenario']==sc)&(df['Year']==y)]
Sank = weave(sdd, dfc, measures='Value', 
      palette=palette).to_widget(**size,margins=dict(left=160, right=160))

In [40]:
Sank.auto_save_png(r'figures/countries/'+c+'_'+sc[:4]+str(y)+'TESankey.png')

SankeyWidget(groups=[{'id': 'technologies', 'type': 'process', 'title': 'Technologies', 'nodes': ['technologie…

# Sankeys for TEMBA

In [2]:
df_TEMBA = pd.read_csv(r'C:/Users/KarlaC/MAT-DP/outputs/TEMBA_SANKEY_data.csv')

In [59]:
df_TEMBA.head()

,Material,Region,Scenario,source,target,Unit,Value,Year
0,Aluminium,Central Africa,1.5deg,BECCS,Embodied emissions,Mt CO2,0.0,2015
1,Bentonite,Central Africa,1.5deg,BECCS,Embodied emissions,Mt CO2,0.0,2015
2,Carbon Fiber,Central Africa,1.5deg,BECCS,Embodied emissions,Mt CO2,0.0,2015
3,Cast Iron,Central Africa,1.5deg,BECCS,Embodied emissions,Mt CO2,0.0,2015
4,Cement,Central Africa,1.5deg,BECCS,Embodied emissions,Mt CO2,0.0,2015


In [79]:
#### DEFINE ALL SLICES IN ORDERING AS VARIABLES

materials = ['Aluminium', 'Bentonite', 'Carbon Fiber',
            'Cast Iron', 'Cement','Ceramics', 'Concrete','Copper', 'Epoxy', 'EVA ', 'Fibre Glass',
            'Glass', 'Lubricant', 'Non-Ferrous Metal', 'Paint', 'Plastic', 'PVC','Resin', 'Sand',
            'Silicon', 'Steel', 'Stainless Steel']

technologies = ['Wind (Onshore)', 'Wind (Offshore)',
       'Solar CSP', 'Solar PV', 'Hydro', 'Oil','Gas', 'Gas CCS', 'Nuclear',
       'Geothermal', 'Coal', 'Coal CCS', 'Biomass', 'BECCS', 'Hydrogen']
regions = ['Central Africa', 'Eastern Africa', 'Northern Africa',
       'Southern Africa', 'Western Africa']

emissions = ['Material embodied emissions']

## Technologies to materials

In [44]:
partition_type = Partition.Simple('Material',df['Material'].unique())
nodes = {'technologies': ProcessGroup(technologies, Partition.Simple('process', technologies), title='Technologies'),
         'materials': ProcessGroup(materials, Partition.Simple('process', materials), title='Materials')
         }                         

ordering = [
    ['technologies'],
     ['materials']
    ]

bundles = [
    Bundle('technologies','materials')
]

colors = ['rgb(0.961, 1.034, 1.084)',
#           'rgb(245, 237, 226)',
        'rgb(179, 154, 154)',
        'rgb(220, 142, 106)',
        'rgb(232, 218, 120)',
        'rgb(162, 77, 55)',
        'rgb(187, 208, 182)',
        'rgb(61, 50, 25)',
        'rgb(195, 127, 120)',
        'rgb(108, 139, 124)',
        'rgb(183, 116, 87)',
        'rgb(255, 253, 228)',
        'rgb(155, 172, 162)',
        'rgb(213, 219, 139)',
        'rgb(32, 64, 79)',
        'rgb(172, 162, 151)',
        'rgb(222, 215, 199)',
        'rgb(213, 237, 231)',
        'rgb(87, 207, 197)',
        'rgb(91, 110, 95)',
        'rgb(251, 205, 198)',
        'rgb(209, 211, 212)',
        'rgb(182, 198, 226)'
            ]

# colors =['rgb(0.961, 1.034, 1.084)',
#  'rbg(0.702, 0.604, 0.604)',
#  '(0.863, 0.557, 0.416)',
# '(0.91, 0.855, 0.471)',
#  '(0.635, 0.302, 0.216)',
# '(0.733, 0.816, 0.714)',
# '(0.239, 0.196, 0.098)',
# '(0.765, 0.498, 0.471)',
#  '(0.424, 0.545, 0.486)',
# '(0.718, 0.455, 0.341)',
#  '(1.0, 0.992, 0.894)',
# '(0.608, 0.675, 0.635)',
#  '(0.835, 0.859, 0.545)',
#  '(0.125, 0.251, 0.31)',
# '(0.675, 0.635, 0.592)',
#  '(0.871, 0.843, 0.78)',
#  '(0.835, 0.929, 0.906)',
#  '(0.341, 0.812, 0.773)',
#  '(0.357, 0.431, 0.373)',
#  '(0.984, 0.804, 0.776)',
#  '(0.82, 0.827, 0.831)',
#  '(0.714, 0.776, 0.886)']

palette = pd.DataFrame(list(zip(materials,colors))).set_index(0)[1].to_dict()

sdd = SankeyDefinition(nodes, bundles, ordering,flow_partition=partition_type)
size = dict(width=1000, height=700)

In [ ]:
# Floweaver can't save in loop
# df = df.rename(columns = {'Source':'source','Target':'target'})
# for c in ['UK','Uganda']:
#     for sc in df[df['Country']==c]['Scenario'].unique():
#         for y in df[(df['Country']==c)&(df['Scenario']==sc)]['Year'].unique():
#             dfc = df[(df['Country']==c)&(df['Scenario']==sc)&(df['Year']==y)]
#             print(c,sc, y)
#             Sank = weave(sdd, dfc, measures='Value', 
#                   palette=palette).to_widget(**size,margins=dict(left=100, right=120))
         

In [ ]:
df_TEMBA = df_TEMBA.rename(columns = {'Source':'source','Target':'target'})
# r = 'Central Africa'
sc = 'Reference'
y = 2010
dfc = df_TEMBA[(df_TEMBA['Region']==r)&(df_TEMBA['Scenario']==sc)&(df_TEMBA['Year']==y)]
Sank = weave(sdd, dfc, measures='Value', 
      palette=palette).to_widget(**size,margins=dict(left=160, right=160))

In [ ]:
Sank.auto_save_png(r'figures/TEMBA/'+r+'_'+sc+str(y)+'TMSankey.png')

## Technologies to emissions

In [85]:
partition_type = Partition.Simple('Material',df_TEMBA['Material'].unique())
partition_otype = Partition.Simple('Region', df_TEMBA['Region'].unique())
nodes = {'technologies': ProcessGroup(technologies, Partition.Simple('process', technologies), title='Technologies'),
#          'regions': ProcessGroup(regions,partition_otype, title='Region'),
        'emissions': ProcessGroup(emissions, Partition.Simple('process', emissions), title='Embodied Emissions'),
         'region': Waypoint(partition_otype, title = 'Region')
         }                         
# Partition.Simple('process', regions)

ordering = [
    ['technologies'],
    ['region'],
     ['emissions']
    ]

bundles = [
    Bundle('technologies', 'emissions', waypoints=['region']),
#     Bundle('regions', 'emissions')
]

colors = ['rgb(0.961, 1.034, 1.084)',
#           'rgb(245, 237, 226)',
        'rgb(179, 154, 154)',
        'rgb(220, 142, 106)',
        'rgb(232, 218, 120)',
        'rgb(162, 77, 55)',
        'rgb(187, 208, 182)',
        'rgb(61, 50, 25)',
        'rgb(195, 127, 120)',
        'rgb(108, 139, 124)',
        'rgb(183, 116, 87)',
        'rgb(255, 253, 228)',
        'rgb(155, 172, 162)',
        'rgb(213, 219, 139)',
        'rgb(32, 64, 79)',
        'rgb(172, 162, 151)',
        'rgb(222, 215, 199)',
        'rgb(213, 237, 231)',
        'rgb(87, 207, 197)',
        'rgb(91, 110, 95)',
        'rgb(251, 205, 198)',
        'rgb(209, 211, 212)',
        'rgb(182, 198, 226)'
            ]

palette = pd.DataFrame(list(zip(materials,colors))).set_index(0)[1].to_dict()

sdd = SankeyDefinition(nodes, bundles, ordering,flow_partition=partition_type)
size = dict(width=1000, height=700)

In [ ]:
# for c in ['UK','Uganda']:
#     for sc in df[df['Country']==c]['Scenario'].unique():
#         for y in df[(df['Country']==c)&(df['Scenario']==sc)]['Year'].unique():
#             dfc = df[(df['Country']==c)&(df['Scenario']==sc)&(df['Year']==y)]
#             print(c,sc, y)

In [87]:
df_TEMBA = df_TEMBA.rename(columns = {'Source':'source','Target':'target'})
# r = 'Central Africa'
sc = 'Reference'
y = 2065
dfc = df_TEMBA[(df_TEMBA['Scenario']==sc)&(df_TEMBA['Year']==y)]
# Sank = 
weave(sdd, dfc, measures='Value', 
      palette=palette).to_widget(**size,margins=dict(left=160, right=160))

SankeyWidget(groups=[{'id': 'technologies', 'type': 'process', 'title': 'Technologies', 'nodes': ['technologie…

In [77]:
Sank.auto_save_png(r'figures/TEMBA/'+'_'+sc[:4]+str(y)+'TESankey.png')

In [ ]:
# example from Leo's Sankey below
# http://localhost:8889/notebooks/Dropbox%20(Cambridge%20University)/Leonardo%20PhD/3rd%20Chapter/IEA%20Future%20Scenario/SankeyETP-2014(KCB2020).ipynb

In [ ]:
#### DEFINE ALL SLICES IN ORDERING AS VARIABLES

#inputs = slice 0

# primaryenergy=['Oil', 'Coal',  'Nuclear', 'Biomass and waste',
#        'Renewable', 'Hydrogen', 'Other','Natural gas','Ambient gain']

# transform =['Fuel Processing','Power Plants']

# sectors = ['Road-Light','Road-Heavy','Aviation', 'Navigation','Rail','Agriculture', 'Industry', 'Residential', 'Services']


# slice_2 = ['Mechanical', 'Process Heating Indirect', 
#        'Process Heating Direct', 'Hot Water','Space Heating','Process Cooling', 'Space Cooling',
#         'Information', 'Illumination']

# devices = ['Spark Ignition Engine','Fuel Cell', 'Diesel Engine','Gas Turbine', 'Electric Motor',
#            'Boiler', 'Burner','Heat Pump','Electric Heater', 'Cooler',
#          'Electronics', 'Light Device',
#         ]

# useful = ['Work','Thermal' , 'Information','loss']

In [ ]:

# partition_type = Partition.Simple('material',df['material'].unique())

# nodes = {
#     'primary':ProcessGroup(primaryenergy),
#     'transform': ProcessGroup(transform),
#     'sector': ProcessGroup(sectors),
#     'device': ProcessGroup(devices),
#     'useful': ProcessGroup(useful),
    
# }

# # Partition
# nodes['primary'].partition =Partition.Simple('process',primaryenergy)

# nodes['transform'].partition =Partition.Simple('process',transform)
# nodes['sector'].partition =Partition.Simple('process',sectors)
# nodes['device'].partition =Partition.Simple('process',devices)
# nodes['useful'].partition =Partition.Simple('process',useful)

# #Waypoints


# # Order
# ordering = [
#     ['primary'],
#     ['transform'],
#     ['sector' ],
#     ['device' ],
#     ['useful']
# ]

# palette_leo={'Coal':'black',
#              'Nuclear':'purple',
#              'Natural gas':'blue',
#             'Electricity':'yellow',
#              'Heat':'orange',
#             'Oil products':'brown',
#              'Oil':'brown',
#              'Renewable':'green',
#             'Biomass, waste and other renewables':'green',
#             'Biomass and waste':'green'}

# bundles = [ 
#     Bundle('primary', 'transform'),
#     Bundle('transform', 'sector'),
#     Bundle('sector', 'device'),
#     Bundle('device', 'useful')
#            ]

# sdd = SankeyDefinition(nodes, bundles, ordering,flow_partition=partition_type)
# size = dict(width=1000, height=600)

In [ ]:

# weave(sdd, df,measures='energy',palette=palette_leo).to_widget(**size)
